####    In this notebook I used two 'efficientnet-b4' Neural Networks (NN). First one was trained on each separate image. Second NN was trained using 6 given images as 6 dimension input. This way I got 7 predictions for each observation (first NN gives 6 predictions (since each observation includes 6 seperate images) and second NN gives one prediction (since it uses the same observtion but considering all 6 pictures as 6 dimnsion input)).¶

####    After that I created DataFrame whre each column is prediction of NN. So I got 7 columns of predictions and 8th column is target. Then RandomForest and CatBoost Classifier used this DataFrame for training. A simple average of RandomForest and CatBoost Classifier was used as a final answer.

1. [First NN](#section-one)
  - [Training](#section-oneB)
2. [Second NN](#section-two)
  - [Training](#section-twoB)
3. [Random Forest ](#section-three)
4. [CatBoostClassifier](#section-four)
5. [Composition and submission](#section-five)


In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torch
import tqdm
import copy
import cv2
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os
import pandas as pd
import seaborn as sns
#import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import StepLR

## First NN
<a id="section-one"></a>

In [ ]:
#prepare data for training first NN
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
X = df['img_path'].values
Y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.05, stratify = Y, random_state = 1)

df = pd.DataFrame({'image': X_train,
                      'target': y_train})
df = df.sample(frac=1)

<a id="section-one"></a>

In [ ]:
class make_tensor:
    
    def __init__(self, images, labels, mode = 'train'):
        self.image = images
        self.label = labels
        self.mode = mode
        
    def __len__(self):
        return len(self.image)

    def __getitem__(self, idx):
        im = self.image[idx]
        lb = self.label[idx]
        sample = {"image":  torch.tensor(im, dtype=torch.float), 
                  "label":  torch.tensor(lb).item()}
            
        return sample

In [ ]:
X = []
Y = []

for path, target in zip(X_test, y_test): 
    temp = np.load(path)
    if target == 1:
        for t in range(6):
            label = [1,0][t % 2]
            im = temp[t].astype(np.float32)
            im = cv2.resize(im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
            im1 = cv2.flip(im, 1)
                        
            X.append(im)
            X.append(im1)
            Y.append(label)
            Y.append(label)
        
    else:
         for t in range(6):
                label = 0
                im = temp[t].astype(np.float32)
                im = cv2.resize(im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                        
                X.append(im)
                Y.append(label)
    del(temp)

print(np.shape(X))
valid_data = make_tensor(X, Y)
valid_data = DataLoader(valid_data, batch_size=64, shuffle=True, drop_last = True)

del(X, Y)

In [ ]:
class CNN(nn.Module):
    def __init__(self, name, output_dim):
        super(CNN, self).__init__()
        self.model = EfficientNet.from_pretrained(name)
        self.model._fc = nn.Linear(in_features = self.model._fc.in_features, out_features=output_dim)
        self.conv =  nn.Conv2d(1, 3, kernel_size=1, stride=1, bias=False)
        
        
    def forward(self, image):
        image.requires_grad_(True)
        image = torch.unsqueeze(image, 1)
        image = self.conv(image)
        
        output = self.model(image)
        
        return(output)

In [ ]:
model = CNN(name = 'efficientnet-b4', output_dim = 1)

#### Training of the first NN model
<a id="section-oneB"></a>


In [ ]:
best_model_wts = []
auc_fin_tr = []
acc_fin_tr = []
auc_fin_val = []
acc_fin_val = []
los_fin_train = []
los_fin_test = []

def train(df, x_valid, model, learning_rate = 0.0001, num_epochs = 1, max_num = 512):
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    scheduler = StepLR(optimizer, step_size = 2, gamma=0.1, verbose=True)
    

    best_auc = 0.0
    auc_tr_eph = []
    acc_tr_eph = []    
        
    for epoch in tqdm.tqdm(range(num_epochs), position=0, leave=True, desc='Epoch'):
        
        running_loss_tr = 0.0
        running_loss_test = 0.0
        
        acc_tr = []
        auc_tr = []
        
        start = -max_num
        end = 0
        
        model.train()
        
        for batch_counter in tqdm.tqdm(range(int(len(df)/max_num)),position=0, leave=True, desc='Load batch'):
            #print("Batch counter:", batch_counter)
            start += max_num
            end += max_num
            X = []
            Y = [] 
            auc_tr_mb = []
            acc_tr_mb = []
        
            for path, target in zip(df['image'][start:end], df['target'][start:end].values): 
                temp = np.load(path)
                if target == 1:
                    for t in range(6):
                        label = [1,0][t % 2]
                        im = temp[t].astype(np.float32)
                        im = cv2.resize(im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                        im1 = cv2.flip(im, 1)
                        
                        X.append(im)
                        X.append(im1)
                        Y.append(label)
                        Y.append(label)
        
                else:
                    for t in range(6):
                        label = 0
                        im = temp[t].astype(np.float32)
                        im = cv2.resize(im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                        
                        X.append(im)
                        Y.append(label)
        
        
            
            x_train = make_tensor(X,Y)
            x_oversam = [x['image'].numpy() for x in x_train]
            y_oversam = [x['label'] for x in x_train]
            
            n, xd, yd = np.shape(x_oversam)
            #print(n)
            y_oversam = np.reshape(y_oversam, (-1, 1))
            x_oversam = np.reshape(x_oversam, (n,xd*yd))
            
            overrsample = RandomOverSampler(sampling_strategy=1)
            X, Y = overrsample.fit_resample(x_oversam, y_oversam)
            n, _ = np.shape(X)
            #print(n)
            X = np.reshape(X, (n, xd, yd))
            Y = np.reshape(Y, (-1,))
            
            
            x_train = make_tensor(X,Y)
            x_train = DataLoader(x_train, batch_size=32, shuffle=True, drop_last = True)
            del(X, Y)
            del(x_oversam, y_oversam)
        
            for data in tqdm.tqdm(x_train, position=0, leave=True, desc='Training'):
                model.zero_grad()
                x = data['image']
                y = data['label']
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.float)
        
                output =  model(x)
            #print('Выход обучения: ', output[:3] )
                loss = loss_fn(output, torch.unsqueeze(y, 1))
                loss.backward()
                optimizer.step()
                running_loss_tr += loss.item()
            
                y = torch.unsqueeze(y, 1).detach().cpu().numpy().tolist()
                output = output.detach().cpu().numpy().tolist()
                output = sum(output, [])
            
                for i in range(len(output)):
                    if output[i] >= 0:
                        output[i] = int(1)
                    else:
                        output[i] = int(0)
                
                roc = roc_auc_score(y, output)
                acc = accuracy_score(y, output)
                auc_tr_mb.append(roc)
                acc_tr_mb.append(acc)
                                     
            
            auc_tr.append(np.mean(auc_tr_mb))
            acc_tr.append(np.mean(acc_tr_mb))
        
        auc_tr_eph = np.mean(auc_tr)
        acc_tr_eph = np.mean(acc_tr)
    
        del(auc_tr, acc_tr)
        
        auc_val = []
        acc_val = []
    
        model.eval()
        with torch.no_grad():
            
            
            for data in tqdm.tqdm(x_valid, position=0, leave=True, desc='Testing'):
                x = data['image']
                y = data['label']
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.float)
            
                output =  model(x)
                loss = loss_fn(output, torch.unsqueeze(y, 1))
                running_loss_test += loss.item()
                y = torch.unsqueeze(y, 1).detach().cpu().numpy().tolist()
                output = output.detach().cpu().numpy().tolist()
                output = sum(output, [])
            
                for i in range(len(output)):
                    if output[i] >= 0:
                        output[i] = int(1)
                    else:
                        output[i] = int(0)
                
                try:
                    auc_val.append(roc_auc_score(y, output))
                    acc_val.append(accuracy_score(y, output))
                except ValueError:
                    pass

            auc_val = np.mean(auc_val)
            acc_val = np.mean(acc_val)
         
        if best_auc < auc_val:
            best_auc = auc_val
            torch.save(model.state_dict(), f'./best_us_model-epoch-{epoch+1}.pth')
            print("This model is saved.")
    
    
        auc_fin_tr.append(auc_tr_eph)
        acc_fin_tr.append(acc_tr_eph)
        auc_fin_val.append(auc_val)
        acc_fin_val.append(acc_val)
        los_fin_train.append(running_loss_tr)
        los_fin_test.append(running_loss_test)
        
        print(f'Epoch: {epoch} \n  Train loses = {running_loss_tr}, \n Train accur = {acc_tr_eph}, \n \
        Train AUC = {auc_tr_eph} \n Val loses = {running_loss_test}, \n \
        Val accur = {acc_val} \n Val AUC = {auc_val}, ') 
        scheduler.step()

In [ ]:
#train(df, valid_data, model = model, learning_rate = 0.001, num_epochs = 5, max_num = 256)

In [ ]:
model.load_state_dict(torch.load('../input/usmodel5epoch/new_SETI_us_5epoch.pth'))

## Second NN
<a id="section-two"></a>

In [ ]:
#prepare data for training second NN
overrsample = RandomOverSampler(sampling_strategy=1)
X_train, y_train = overrsample.fit_resample(X_train.reshape(-1,1), y_train)
X_train = X_train.reshape(-1,)

df1 = pd.DataFrame({'image': X_train,
                      'target': y_train})
df1 = df1.sample(frac=1)

In [ ]:
class make_tensor1:
    
    def __init__(self, images, labels):
        self.image = images
        self.label = labels
        
    def __len__(self):
        return len(self.image)

    def __getitem__(self, idx):
        im = np.load(self.image[idx])
        lb = self.label[idx]
        sample = {"image":  torch.tensor(im, dtype=torch.float), 
                  "label":  torch.tensor(lb).item()}
            
        return sample

In [ ]:
valid_data1 = make_tensor1(X_test, y_test)
valid_data1 = DataLoader(valid_data1, batch_size=64, shuffle=True, drop_last = True)

In [ ]:
class common_CNN(nn.Module):
    def __init__(self, name, output_dim):
        super(common_CNN, self).__init__()
        self.model = EfficientNet.from_pretrained(name)
        self.model._fc = nn.Linear(in_features = self.model._fc.in_features, out_features=output_dim)
        self.conv =  nn.Conv2d(6, 3, kernel_size=1, stride=1, bias=False)
        
        
    def forward(self, image):
        image.requires_grad_(True)
        #image = torch.unsqueeze(image, 1)
        image = self.conv(image)
        
        output = self.model(image)
        
        return(output)

In [ ]:
model1 = common_CNN(name = 'efficientnet-b4', output_dim = 1)

#### Training of the first NN model
<a id="section-twoB"></a>


In [ ]:
best_model_wts = []
auc_fin_tr = []
acc_fin_tr = []
auc_fin_val = []
acc_fin_val = []
los_fin_test = []


def common_train(df,x_valid, model, epoch = 1, max_num = 256, learning_rate = 0.0001):
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    scheduler = StepLR(optimizer, step_size = 1, gamma=0.1, verbose=True) 
    best_auc = 0
    
    for ep in range(epoch):
        start = -max_num
        end = 0
        auc_tr = []
        acc_tr = []
    
        for batch_counter in tqdm.tqdm(range(int(len(df)/max_num)),position=0, leave=True, desc='Load batch'):
            #print("Batch counter:", batch_counter, 'from', int(len(df)/max_num))
            start += max_num
            end += max_num
            
            x = df['image'][start:end].values 
            y = df['target'][start:end].values
                
            x_train = make_tensor1(x,y)
            x_train = DataLoader(x_train, batch_size=32, shuffle=True, drop_last = True)
            del(x, y)
            
            auc_tr_mb = []
            acc_tr_mb = []
            
            model.train()
            for data in tqdm.tqdm(x_train, position=0, leave=True, desc='Training'):
                model.zero_grad()
                x = data['image']
                y = data['label']
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.float)
        
                output =  model(x)
                loss = loss_fn(output, torch.unsqueeze(y, 1))
                loss.backward()
                optimizer.step()
            
                y = torch.unsqueeze(y, 1).detach().cpu().numpy().tolist()
                output = output.detach().cpu().numpy().tolist()
                output = sum(output, [])
            
                for i in range(len(output)):
                    if output[i] >= 0:
                        output[i] = int(1)
                    else:
                        output[i] = int(0)
                 
                try:
                    auc_tr_mb.append(roc_auc_score(y, output))
                    acc_tr_mb.append(accuracy_score(y, output))
                except ValueError:
                    pass
                                     
            
            auc_tr.append(np.mean(auc_tr_mb))
            acc_tr.append(np.mean(acc_tr_mb))
        
        auc_tr_eph = np.mean(auc_tr)
        acc_tr_eph = np.mean(acc_tr)
    
        del(auc_tr, acc_tr)
        
        auc_val = []
        acc_val = []
        running_loss_test = 0
    
        model.eval()
        with torch.no_grad():   
            for data in tqdm.tqdm(x_valid, position=0, leave=True, desc='Testing'):
                x = data['image']
                y = data['label']
                x = x.to(device, dtype=torch.float)
                y = y.to(device, dtype=torch.float)
            
                output =  model(x)
                #print('Выход валидации: ', output[:3] )
                loss = loss_fn(output, torch.unsqueeze(y, 1))
                running_loss_test += loss.item()
                y = torch.unsqueeze(y, 1).detach().cpu().numpy().tolist()
                output = output.detach().cpu().numpy().tolist()
                output = sum(output, [])
            
                for i in range(len(output)):
                    if output[i] >= 0:
                        output[i] = int(1)
                    else:
                        output[i] = int(0)
                
                try:
                    auc_val.append(roc_auc_score(y, output))
                    acc_val.append(accuracy_score(y, output))
                except ValueError:
                    pass
        
            auc_val = np.mean(auc_val)
            acc_val = np.mean(acc_val)
         
        if best_auc < auc_val:
            best_auc = auc_val
            torch.save(model.state_dict(), f'./com_model_epoh{ep+6}.pth')
            print("This model is saved.")
    
    
        auc_fin_tr.append(auc_tr_eph)
        acc_fin_tr.append(acc_tr_eph)
        auc_fin_val.append(auc_val)
        acc_fin_val.append(acc_val)
        los_fin_test.append(running_loss_test)
        
        print(f'Epoch: {ep+1} \n Train accur = {acc_tr_eph}, \n \
        Train AUC = {auc_tr_eph} \n Val loses = {running_loss_test}, \n \
        Val accur = {acc_val} \n Val AUC = {auc_val}') 
        scheduler.step()


In [ ]:
#common_train(df = df1,x_valid = valid_data1, model = model1, learning_rate = 0.0001, epoch = 1, max_num = 512)

In [ ]:
model1.load_state_dict(torch.load('../input/com-model-epoch1/com_model_epoh1.pth'))

## Random Forest prediction
<a id="section-three"></a>

In [ ]:
#Create Data Frame where each column is prediction of first and second NN
feature0 = []
feature1 = []
feature2 = []
feature3 = []
feature4 = []
feature5 = []
feature6 = []
target = []

def feature_table(x, y, model, model1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model1.to(device)
    
    valid = make_tensor1(x, y)
    valid = DataLoader(valid, batch_size=64, shuffle=False, drop_last = False)
    
    with torch.no_grad():
        model.eval()
        model1.eval()
        for data in valid:
            
            x = data["image"]
            y = data['label']
            x = x.to(device, dtype=torch.float)
            y = y.to(device, dtype=torch.float)
            target.extend(y.tolist())
            
            output =  model1(x)
                
            output = torch.sigmoid(output)
            output = output.reshape(-1,) 
            feature0.extend(output.tolist())
            
            
            for im in x:
                temp = []
                for sub_im in im:
                    sep_im = sub_im.cpu().numpy().astype(np.float32)
                    sep_im = cv2.resize(src = sep_im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                    temp.append(sep_im)
                    
                in_put = make_tensor(temp, y[range(6)])
                in_put = DataLoader(in_put, batch_size=6, shuffle=False, drop_last = False)
                    #sep_im = torch.tensor(sep_im, dtype=torch.float)
                    #sep_im  = torch.unsqueeze(sep_im , 0)
                    
                out = model(next(iter(in_put))['image'].to(device, dtype=torch.float))
                out = out.reshape(-1,) 
                out = torch.sigmoid(out)
                for n in range(6):
                    globals()["feature{}".format(n+1)].append(out[n].item())
            
    
    feature_df1 = pd.DataFrame(data= zip(feature1, feature2, feature3, feature4, feature5,
                                      feature6, target),columns=['feature1', 'feature2','feature3', 
                                                          'feature4', 'feature5', 'feature6', 'target'])
    
    feature_df1.insert(0, value = feature0, column = 'feature0')
    
    return feature_df1

In [ ]:
#feature_df = feature_table(x = X_test, y = y_test, model = model, model1 = model1)

In [ ]:
feature_df = pd.read_csv('../input/feature-df-5us-1com/feature_df_5us_1com.csv')

In [ ]:
#Use GridSearch to find best parametrs for Random Forest Classifier

param_grid = { 
    'n_estimators': [150, 350],
    'max_features': [4,5,6,7],
    'max_depth' : [5, 10, 15, None],
    'min_samples_split': [2, 3, 7, 10],
    'bootstrap' : [True],
    'criterion' :['gini', 'entropy']
}


#rf = RandomForestClassifier(random_state=0, n_jobs = -1)
#clf = GridSearchCV(rf, param_grid, cv = 3, refit='AUC')
#clf.fit(feature_df.iloc[:, 0:7], feature_df['target'])


In [ ]:
import pickle
clf = pickle.load(open('../input/forest-5us-1com/forest_5us_1com.sav', 'rb'))

In [ ]:
cm = confusion_matrix(feature_df.target, clf.predict(feature_df.iloc[:, 0:7]))
cm

In [ ]:
roc_auc_score(feature_df.target, clf.predict(feature_df.iloc[:, 0:7]))

## CatBoost
<a id="section-four"></a>

In [ ]:
from catboost import CatBoostClassifier
catb = CatBoostClassifier(random_state=42,
                                 thread_count=4,
                                 verbose=False,
                                 loss_function='Logloss',
                                 od_type="Iter",
                                 early_stopping_rounds=500,
                                 iterations=5000)
catb.fit(feature_df.iloc[:, 0:7], feature_df['target'])

## SUBMISSION
<a id="section-five"></a>

prepare data for submission

In [ ]:
#prepare data for submission
path = []
id = []
arr = os.listdir('../input/seti-breakthrough-listen/test')
for folder in arr:
    content = os.listdir(f'../input/seti-breakthrough-listen/test/{folder}')
    for t in range(len(content)):
        path.append(f'../input/seti-breakthrough-listen/test/{folder}/{content[t]}')
        id.append(content[t][:-4])
submission = pd.DataFrame(data = zip(id, path), columns = ['id','im_path'])

In [ ]:
feature0_sub = []
feature1_sub = []
feature2_sub = []
feature3_sub = []
feature4_sub = []
feature5_sub = []
feature6_sub = []

def feature_table_sub(x, model, model1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model1.to(device)
    y = np.zeros(len(submission))
    
    valid = make_tensor1(x, y)
    valid = DataLoader(valid, batch_size=64, shuffle=False, drop_last = False)
    
    with torch.no_grad():
        model.eval()
        model1.eval()
        for data in valid:
            
            x = data["image"]
            x = x.to(device, dtype=torch.float)
            
            output =  model1(x)
                
            output = torch.sigmoid(output)
            output = output.reshape(-1,) 
            feature0_sub.extend(output.tolist())
            
            
            for im in x:
                temp = []
                for sub_im in im:
                    sep_im = sub_im.cpu().numpy().astype(np.float32)
                    sep_im = cv2.resize(src = sep_im, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
                    temp.append(sep_im)
                
                in_put = make_tensor(temp, y[range(6)])
                in_put = DataLoader(in_put, batch_size=6, shuffle=False, drop_last = False)
                    #sep_im = torch.tensor(sep_im, dtype=torch.float)
                    #sep_im  = torch.unsqueeze(sep_im , 0)
                
                out = model(next(iter(in_put))['image'].to(device, dtype=torch.float))
                out = out.reshape(-1,) 
                out = torch.sigmoid(out)
                for n in range(6):
                    globals()["feature{}_sub".format(n+1)].append(out[n].item())
            
    
    feature_df1_sub = pd.DataFrame(data= zip(feature1_sub, feature2_sub, feature3_sub, feature4_sub,
                                             feature5_sub,feature6_sub),columns=['feature1', 'feature2','feature3', 
                                                          'feature4', 'feature5', 'feature6'])
    
    feature_df1_sub.insert(0, value = feature0_sub, column = 'feature0')
    
    return feature_df1_sub

In [ ]:
#feature_df_sub = feature_table_sub(submission.im_path, model = model, model1 = model1)

In [ ]:
feature_df_sub = pd.read_csv('../input/df-sub-5us-1-com/df_sub_5us_1com.csv')

In [ ]:
feature_df_sub

In [ ]:
submis1 = catb.predict_proba(feature_df_sub.iloc[:, 0:7])[:,1]
submis2 = clf.predict_proba(feature_df_sub.iloc[:, 0:7])[:,1]
submis = (submis1 + submis2)/2

sub_cat_forest_5us_1com = pd.DataFrame({'id': submission.id,
                    'target' : submis2})
sub_cat_forest_5us_1com.to_csv('forest_5us_1com.csv', index = False)